In [74]:
from pwn import *

In [132]:
r = remote('edu-ctf.csie.org', 42070)
cipher = r.recvline()
cipher = cipher.split(b"cipher = ")[-1][:-1].decode()
print(cipher)
cipher = bytes.fromhex(cipher)
print(cipher)

[x] Opening connection to edu-ctf.csie.org on port 42070
[x] Opening connection to edu-ctf.csie.org on port 42070: Trying 140.112.31.97
[+] Opening connection to edu-ctf.csie.org on port 42070: Done
a755e5cbfee8e9580d64499aeeca630cf2c65dd859fe23f6a87331114f7a6083527f712534d0e4d26d56256d1fc9fcd4
b'\xa7U\xe5\xcb\xfe\xe8\xe9X\rdI\x9a\xee\xcac\x0c\xf2\xc6]\xd8Y\xfe#\xf6\xa8s1\x11Oz`\x83R\x7fq%4\xd0\xe4\xd2mV%m\x1f\xc9\xfc\xd4'


In [135]:
def xor(x, y):
    return bytes(a ^ b for a, b in zip(x, y))

for blk in range((len(cipher) - 16) // 16):
    blk_data = cipher[blk * 16 + 16 : (blk + 1) * 16 + 16]
    prev_blk_data = cipher[blk * 16 : (blk + 1) * 16]
    
    ok = b''
    for i in range(16): # crack iv's bit
        print(i)
        for c in range(256): # iterate all possible characters
            iv = b'\x00' * (15 - i) + bytes([c]) + ok
            p = iv + blk_data
            p = p.hex().encode()
            r.sendlineafter(b'cipher = ', p)
            q = r.recvline()
            if q != b'NOOOOOOOOO\n':
                ok = bytes([c ^ 0x80]) + ok
                print(xor(prev_blk_data[-i-1:], ok))
                break

0
b'9'
1
b'79'
2
b'479'
3
b'a479'
4
b'fa479'
5
b'ffa479'
6
b'7ffa479'
7
b'37ffa479'
8
b'f37ffa479'
9
b'9f37ffa479'
10
b'd9f37ffa479'
11
b'{d9f37ffa479'
12
b'G{d9f37ffa479'
13
b'AG{d9f37ffa479'
14
b'LAG{d9f37ffa479'
15
b'FLAG{d9f37ffa479'
0
b'\x00'
1
b'\x00\x00'
2
b'\x00\x00\x00'
3
b'\x00\x00\x00\x00'
4
b'\x00\x00\x00\x00\x00'
5
b'\x00\x00\x00\x00\x00\x00'
6
b'\x00\x00\x00\x00\x00\x00\x00'
7
b'\x00\x00\x00\x00\x00\x00\x00\x00'
8
b'\x80\x00\x00\x00\x00\x00\x00\x00\x00'
9
b'\n\x80\x00\x00\x00\x00\x00\x00\x00\x00'
10
b'}\n\x80\x00\x00\x00\x00\x00\x00\x00\x00'
11
b'f}\n\x80\x00\x00\x00\x00\x00\x00\x00\x00'
12
b'ff}\n\x80\x00\x00\x00\x00\x00\x00\x00\x00'
13
b'7ff}\n\x80\x00\x00\x00\x00\x00\x00\x00\x00'
14
b'47ff}\n\x80\x00\x00\x00\x00\x00\x00\x00\x00'
15
b'f47ff}\n\x80\x00\x00\x00\x00\x00\x00\x00\x00'
